In [1]:
import numpy as np
import scipy.stats as stats
import altair as alt
import pandas as pd

def simulate_ebm(N, J):
    """
    Simulate an Event-Based Model (EBM) for disease progression.
    
    Args:
    N (int): Number of biomarkers.
    J (int): Number of participants.
    
    Returns:
    tuple: A tuple containing:
        - S (numpy.ndarray): The random permutation representing the order of biomarker implication.
        - kjs (numpy.ndarray): The disease stages for participant.
        - X (numpy.ndarray): The biomarker matrix with participant data.
          - Each cell in X is tuple containing participantId, biomarker value, 
            disease stage of this participant, disease stage current biomarker indicates
            and healthy status
    """
    
    # Random permutation for ordering biomarkers, starting from 0
    # S indicates the disease progression, S[0] -> stage1, S[1] -> stage2
    S = np.random.permutation(N)
    
    # Generate a random stage for each participant
    # The stage should be between 0 and N, inclusive
    kjs = np.random.randint(0, N+1, size=J)
    
    # Initiate biomarker matrix (J participants x N biomarkers), with entries as None
    X = np.full((J, N), None, dtype=object)

    # Generate theta and phi, which are both a function of biomarker n
    # np.random.seed(42)
    # theta_means = {n: np.random.randint(low=0, high=9) for n in range(N)}
    # theta_vars = {n: np.random.rand() for n in range(N)}
    # phi_means = {n: np.random.randint(low=0, high=9) for n in range(N)}
    # phi_vars = {n: np.random.rand() for n in range(N)}
    theta_means = [1, 3, 5, 6, 8, 0, 4, 2, 7, 9]
    theta_vars = [0.3, 0.5, 0.2, 1.3, 3.3, 2.2, 0.8, 0.9, 0.7, 0.6]
    phi_means = [12, 11, 14, 16, 18, 19, 10, 13, 15, 17]
    phi_vars = [1.3, 2.4, 1.4, 0.9, 1.5, 1.9, 2.4, 1.7, 2.0, 1.0]
    theta = {n: stats.norm(theta_means[n], theta_vars[n]) for n in range(N)}
    phi = {n: stats.norm(phi_means[n], phi_vars[n]) for n in range(N)}

    # Iterate through participants
    for j in range(J):
        # Iterate through biomarkers
        for n in range(N):
            # Disease stage of the current participant
            k_j = kjs[j]
            # Disease stage indicated by the current biomarker
            # Note that biomarkers always indicate that the participant is diseased
            # Thus, S_n >= 1
            S_n = np.where(S == n)[0][0] + 1
            
            # Assign values based on whether the participant's stage is at or past the biomarker's stage
            if k_j >= 1:
                if k_j >= S_n:
                    X[j, n] = (j, theta[n].rvs(), k_j, S_n, 'affected') 
                else:
                    X[j, n] = (j, phi[n].rvs(), k_j, S_n, 'not_affected')  
            else:
                X[j, n] = (j, phi[n].rvs(), k_j, S_n, 'not_affected')        
    return S, kjs, X, theta_means, theta_vars, phi_means, phi_vars

In [2]:
N = 10
J = 100
S, kjs, X, theta_means, theta_vars, phi_means, phi_vars = simulate_ebm(N, J)

In [3]:
theta_means, theta_vars, phi_means, phi_vars

([1, 3, 5, 6, 8, 0, 4, 2, 7, 9],
 [0.3, 0.5, 0.2, 1.3, 3.3, 2.2, 0.8, 0.9, 0.7, 0.6],
 [12, 11, 14, 16, 18, 19, 10, 13, 15, 17],
 [1.3, 2.4, 1.4, 0.9, 1.5, 1.9, 2.4, 1.7, 2.0, 1.0])

In [4]:
df_means_vars = pd.DataFrame([theta_means, theta_vars, phi_means, phi_vars]).transpose()
df_means_vars.columns = ['theta_mean', 'theta_var', 'phi_mean', 'phi_var']
df_means_vars = df_means_vars.rename_axis("biomarker", axis=0).reset_index()
df_means_vars

,biomarker,theta_mean,theta_var,phi_mean,phi_var
0,0,1.0,0.3,12.0,1.3
1,1,3.0,0.5,11.0,2.4
2,2,5.0,0.2,14.0,1.4
3,3,6.0,1.3,16.0,0.9
4,4,8.0,3.3,18.0,1.5
5,5,0.0,2.2,19.0,1.9
6,6,4.0,0.8,10.0,2.4
7,7,2.0,0.9,13.0,1.7
8,8,7.0,0.7,15.0,2.0
9,9,9.0,0.6,17.0,1.0


In [5]:
df_means_vars.to_csv('data/means_vars.csv', index=False)

In [6]:
S

array([8, 3, 5, 0, 9, 2, 6, 4, 7, 1])

In [7]:
X[10]

array([(10, 1.2487920251681248, 9, 4, 'affected'),
       (10, 14.810175172128789, 9, 10, 'not_affected'),
       (10, 4.7498838367405485, 9, 6, 'affected'),
       (10, 6.303474292178, 9, 2, 'affected'),
       (10, 5.438953996958221, 9, 8, 'affected'),
       (10, -1.2447172902921637, 9, 3, 'affected'),
       (10, 4.888399198445938, 9, 7, 'affected'),
       (10, 0.3970552329838919, 9, 9, 'affected'),
       (10, 7.660484744981675, 9, 1, 'affected'),
       (10, 8.752267522620288, 9, 5, 'affected')], dtype=object)

In [8]:
df = pd.DataFrame(X, columns = [f"Biomarker {i}" for i in range(N)])

# make this dataframe wide to long 
df_long = df.melt(var_name = "Biomarker", value_name="Value")

# exapand the value column into a dataframe
values_df = df_long['Value'].apply(pd.Series)
values_df.columns = ['participant', 'measurement', 'k_j', 'S_n', 'affected_or_not']

# join values_df with df_long
df_expanded = df_long.drop('Value', axis = 1).join(values_df)

alt.Chart(df_expanded).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['Biomarker']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "Biomarker:N",
        columns = 5
    ),
    color=alt.Color(
        'Biomarker:N'
    )
).properties(
    width= 140,
    height = 200,
).properties(
    title='Biomarker data for all participants across all stages'
)

alt.Chart(...)

In [9]:
df_expanded.to_csv("data/participant_data.csv", index=False)
df_expanded.head()


,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,Biomarker 0,0,0.481266,9,4,affected
1,Biomarker 0,1,1.276640,5,4,affected
2,Biomarker 0,2,10.055805,3,4,not_affected
3,Biomarker 0,3,12.102054,3,4,not_affected
4,Biomarker 0,4,1.094876,5,4,affected


In [10]:
# get data for conjugate priors
# biomarker 1 & drawn from diseased groups
df_expanded[(
    df_expanded.Biomarker == "Biomarker 1") & (
        df_expanded.affected_or_not == "affected")].to_csv("data/conjugate_data.csv", index=False)

In [11]:
# df_expanded[df_expanded.k_j == 0]

In [12]:
# biomarker data when the participant is at stage 6
df_kj_6 = df_expanded[df_expanded.k_j == 6]
df_kj_6

alt.Chart(df_kj_6).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['Biomarker']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "Biomarker:N",
        columns = 5
    ),
    color=alt.Color(
        'Biomarker:N'
    )
).properties(
    width= 140,
    height = 200,
).properties(
    title='Biomarker data when the participant is at stage six'
)

alt.Chart(...)

In [13]:
# select only biomarker 2 
bio_2_data = df_expanded[df_expanded.Biomarker=='Biomarker 2'].drop(['k_j', 'S_n', 'Biomarker'], axis = 1)
# biomarker2 data, comparing from diseased and healthy groups
alt.Chart(bio_2_data).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['affected_or_not']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "affected_or_not:N",
    ),
    color=alt.Color(
        'affected_or_not:N'
    )
).properties(
    width= 240,
    height = 200,
).properties(
    title='Biomarker2 data, compring healthy group and diseased group'
)

alt.Chart(...)

In [14]:
# participant 10
participant10_data = df_expanded[df_expanded.participant == 10]
alt.Chart(participant10_data).mark_bar().encode(
    x='Biomarker',
    y='measurement',
    color=alt.Color(
        'affected_or_not:N'
    ),
    tooltip=['Biomarker', 'affected_or_not', 'measurement']
).interactive().properties(
    title=f'Biomarker data for participant10 (k_j = {participant10_data.k_j.to_list()[0]})'
)

alt.Chart(...)